In [1]:
try:
    import pylibjpeg
except:
    !mkdir -p /root/.cache/torch/hub/checkpoints/
    !cp ../input/effdet-models/efficientnet_v2s_ra2_288-a6477665.pth /root/.cache/torch/hub/checkpoints/
    !pip install ../input/effdet-models/pydicom-2.3.0-py3-none-any.whl
    !pip install ../input/effdet-models/pylibjpeg-1.4.0-py3-none-any.whl
    !pip install ../input/effdet-models/pylibjpeg_libjpeg-1.3.2-cp37-cp37m-manylinux_2_17_x86_64.manylinux2014_x86_64.whl
    !pip install ../input/effdet-models/python_gdcm-3.0.19-cp37-cp37m-manylinux_2_17_x86_64.manylinux2014_x86_64.whl
    !pip install ../input/effdet-models/pycocotools-2.0.2-cp37-cp37m-linux_x86_64.whl

Processing /kaggle/input/effdet-models/pydicom-2.3.0-py3-none-any.whl
pydicom is already installed with the same version as the provided wheel. Use --force-reinstall to force an installation of the wheel.
Processing /kaggle/input/effdet-models/pylibjpeg-1.4.0-py3-none-any.whl
Processing /kaggle/input/effdet-models/pylibjpeg_libjpeg-1.3.2-cp37-cp37m-manylinux_2_17_x86_64.manylinux2014_x86_64.whl
Processing /kaggle/input/effdet-models/python_gdcm-3.0.19-cp37-cp37m-manylinux_2_17_x86_64.manylinux2014_x86_64.whl
Processing /kaggle/input/effdet-models/pycocotools-2.0.2-cp37-cp37m-linux_x86_64.whl


In [2]:
import pandas as pd
import torch
# import matplotlib.pyplot as plt
# import seaborn as sns
# import ast
import re
import sys

import pydicom as dicom
import pylibjpeg

effdet_path = "../input/effdet-models/effdet"
sys.path.append(effdet_path)


timm_path = "../input/effdet-models/timm-pytorch-image-models"
sys.path.append(timm_path)
import timm
from timm.data import IMAGENET_DEFAULT_MEAN, IMAGENET_DEFAULT_STD
import matplotlib.pyplot as plt
# from matplotlib import patches
# import sklearn
import os
from tqdm import tqdm

omega_path = "../input/effdet-models/omegaconf"
sys.path.append(omega_path)
from omegaconf import OmegaConf

effunet_path = "../input/effdet-models/efficientunet-pytorch-0.0.6"
sys.path.append(effunet_path)

# from effdet import create_model

import glob
# import sklearn
import math
import random
import numpy as np

from PIL import Image

# import cv2
# import albumentations as A
# from albumentations.pytorch import ToTensorV2

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torch import optim
from torchvision import transforms
import torchvision.models as models
import torchvision.transforms as T
import torchvision.transforms.functional as TF
# from transformers import get_cosine_schedule_with_warmup

import warnings

warnings.filterwarnings('ignore')
# from sklearn import metrics, model_selection, preprocessing

# from sklearn.model_selection import GroupKFold


# pos_weight = torch.tensor(pos_weight)
device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

'cuda'

In [3]:
IMAGES_DIR='../input/rsna-2022-cervical-spine-fracture-detection/test_images'
TRAIN_IMAGES_PATH='../input/rsna-2022-cervical-spine-fracture-detection/train_images'
TEST_IMAGES_PATH='../input/rsna-2022-cervical-spine-fracture-detection/test_images'

In [4]:
segmentation_checkpoint="../input/effdet-models/axial_segmentation_effseg_132508-epoch-100.pth"
axial_det_checkpoint="../input/effdet-models/axial_detection_effdet_134352-epoch-52.pth"

In [5]:
# for test, use these
# train_df = pd.read_csv('../input/rsna-2022-cervical-spine-fracture-detection/train.csv').set_index('StudyInstanceUID')
# train_df.head()


# IMAGES_DIR='../input/rsna-2022-cervical-spine-fracture-detection/train_images'
# TRAIN_IMAGES_PATH='../input/rsna-2022-cervical-spine-fracture-detection/train_images'
# TEST_IMAGES_PATH='../input/rsna-2022-cervical-spine-fracture-detection/test_images'


# test_slices = glob.glob(f'{TRAIN_IMAGES_PATH}/1.2.826.0.1.3680043.18*/*')
# test_slices = [re.findall(f'{TRAIN_IMAGES_PATH}/(.*)/(.*).dcm', s)[0] for s in test_slices]
# df_test_slices = pd.DataFrame(data=test_slices, columns=['StudyInstanceUID', 'Slice']).astype({'Slice': int}).sort_values(['StudyInstanceUID', 'Slice']).reset_index(drop=True)
# df_test_slices
# test end

test_slices = glob.glob(f'{TEST_IMAGES_PATH}/*/*')
test_slices = [re.findall(f'{TEST_IMAGES_PATH}/(.*)/(.*).dcm', s)[0] for s in test_slices]
df_test_slices = pd.DataFrame(data=test_slices, columns=['StudyInstanceUID', 'Slice']).astype({'Slice': int}).sort_values(['StudyInstanceUID', 'Slice']).reset_index(drop=True)
df_test_slices

,StudyInstanceUID,Slice
0,1.2.826.0.1.3680043.22327,1
1,1.2.826.0.1.3680043.22327,2
2,1.2.826.0.1.3680043.22327,3
3,1.2.826.0.1.3680043.22327,4
4,1.2.826.0.1.3680043.22327,5
...,...,...
1313,1.2.826.0.1.3680043.5876,454
1314,1.2.826.0.1.3680043.5876,455
1315,1.2.826.0.1.3680043.5876,456
1316,1.2.826.0.1.3680043.5876,457


In [6]:
def rescale_img_to_hu(dcm_ds):
    """Rescales the image to Hounsfield unit.
    """
    return dcm_ds.pixel_array * dcm_ds.RescaleSlope + dcm_ds.RescaleIntercept

def normalize_hu(data):
    # normalize to 0-1
    # return (data - data.min()) / data.max()
    data = np.clip(data, a_min=-2242, a_max=2242) / 4484 + 0.5
    return data

def load_dicom(path):
    """
    This supports loading both regular and compressed JPEG images. 
    See the first sell with `pip install` commands for the necessary dependencies
    """
    ds=dicom.dcmread(path) 

#     img = normalize_hu(rescale_img_to_hu(ds))
    img = rescale_img_to_hu(ds)

    return img, ds.PixelSpacing[0]

# im, pixel_spacing = load_dicom(f'{TRAIN_IMAGES_PATH}/1.2.826.0.1.3680043.10001/1.dcm')
# print(pixel_spacing)
# print(im.shape)
# print(im.min(), im.max())
# print(torch.as_tensor(im).shape)
# plt.figure()
# plt.imshow(im)
# plt.title('regular image')

# im, meta = load_dicom(f'{TRAIN_IMAGES_PATH}/1.2.826.0.1.3680043.10014/1.dcm')
# plt.figure()
# plt.imshow(im)
# plt.title('jpeg')

In [7]:
class DcmDataSet(torch.utils.data.Dataset):    
    def __init__(self, df, path, transforms=None):
        super().__init__()
        self.df = df
        self.path = path
        self.transforms = transforms
        
        self.len = len(self.df)
        
    def __getitem__(self, i):
        
              
        
        try:
            s = self.df.iloc[i]
            prev_s = self.df.iloc[i-1] if i > 0 else s
            prev_s = s if prev_s.StudyInstanceUID != s.StudyInstanceUID else prev_s
            
            next_s = self.df.iloc[i+1] if i < (self.len-1) else s
            next_s = s if next_s.StudyInstanceUID != s.StudyInstanceUID else next_s
            
#             slice = s.Slice
            rpath = os.path.join(self.path, s.StudyInstanceUID, f'{prev_s.Slice}.dcm')
            gpath = os.path.join(self.path, s.StudyInstanceUID, f'{s.Slice}.dcm')
            bpath = os.path.join(self.path, s.StudyInstanceUID, f'{next_s.Slice}.dcm')
            
#             print(rpath)
#             print(os.path.exists(rpath))
            
            g, pixel_spacing = load_dicom(gpath) 
            r, _ = load_dicom(rpath)
            b, _ = load_dicom(bpath)
            
#             print(r.shape)
#             print(g.shape)
#             print(b.shape)
            img = np.stack((r, g, b))
            img = normalize_hu(img)
#             print(img.shape)
#             img = np.transpose(img, (2, 0, 1))  # Pytorch uses (batch, channel, height, width) order. Converting (height, width, channel) -> (channel, height, width)
            if self.transforms is not None:
                img = self.transforms(img)
        except Exception as ex:
            print(ex)
            return None, None
        
        return img, pixel_spacing      
    
    def __len__(self):
        return self.len
    
class DataTransform(nn.Module):
    def __init__(self, image_size=512):
        super().__init__()

        self.image_size = image_size

        self.transform = T.Compose([
            T.Resize((image_size, image_size)),
            T.Normalize(0.5, 0.5),
        ])


    def forward(self, x):
        x = self.transform(torch.as_tensor(x, dtype=torch.float))

        return x

tf = DataTransform()
ds = DcmDataSet(df_test_slices, IMAGES_DIR, tf)
input, pixel_spacing = ds[0]
print(input.shape)
print(input.min(), input.max())
print(pixel_spacing)

# plt.imshow(input.permute(1, 2, 0))

torch.Size([3, 512, 512])
tensor(-0.9135) tensor(0.9692)
0.384766


In [8]:
batch_size = 16
dl = DataLoader(ds, batch_size=batch_size, shuffle=False, num_workers=min(os.cpu_count(), batch_size))

# x, pixel_spacings = next(iter(dl))
# print(x.shape)
# print(pixel_spacings)

In [9]:
from efficientunet import *
def get_axial_segmentation_model(checkpoint):
    model = get_efficientunet_b5(out_channels=2, concat_input=True, pretrained=False)
    
    state = torch.load(checkpoint, map_location=torch.device(device))
    model.load_state_dict(state["model"])
    model.eval()
    return model.to(device)
    
seg_model = get_axial_segmentation_model(segmentation_checkpoint)

In [10]:
from effdet import create_model

def get_axial_detection_model(checkpoint, image_size=512):
    model = create_model('efficientdetv2_ds' , bench_task='predict' , num_classes=1 , image_size=(image_size, image_size), pretrained=False, max_det_per_image=1)

    state = torch.load(checkpoint, map_location=torch.device(device))
    model.load_state_dict(state["model"])
    
    model = model.eval()
    return model.to(device)

det_model = get_axial_detection_model(axial_det_checkpoint)

In [11]:
def get_axial_boundary_from_segmentation(seg, pixel_spacing, throw=100, tol=0.2, max_mm=100):
    """
    seg : H x W
    """
    image_size = seg.shape[0]
    min_size = min(image_size, max_mm / pixel_spacing)
    
    rows, columns = seg.nonzero(as_tuple=True)
    rows.sort()
    columns.sort()
    
    throw = min(len(rows) // 2, throw)
    
    if(len(rows)) == 0:
        return torch.tensor([0, 0, image_size, image_size]).to(device)
    
    xmin, xmax = columns[throw], columns[-throw]
    ymin, ymax = rows[throw], rows[-throw]
    
    w = (xmax - xmin) * (1 + tol)
    h = (ymax - ymax) * (1 + tol)
    new_size = max(w, h, min_size)
    new_size = min(image_size, new_size)
    
    xcenter, ycenter = (xmax + xmin) / 2, (ymax + ymin) / 2
    
    xmin = torch.min(torch.tensor(image_size - new_size), xcenter - new_size / 2)
    xmin = xmin.clip(min=0)
    
    ymin = torch.min(torch.tensor(image_size - new_size), ycenter - new_size / 2)
    ymin = ymin.clip(min=0)
    
    return torch.stack([xmin, ymin, xmin + new_size, ymin + new_size])

In [12]:
def predict_seg(x, model, img_size=256):
    """
    return: N x 1 x H x W
    """
    x = TF.resize(x, (img_size, img_size))
    logits = model(x)

    classification_score, mse_score = logits.sigmoid().chunk(2, dim=1)
    classification_pred = classification_score.gt(0.5).float()
    pred = (classification_pred * mse_score)
    
    return pred

In [13]:
def get_axial_boundary(segs, pixel_spacings, seg_img_size=256):
    boundary_list = []
    for i in range(segs.shape[0]):
        seg = segs[i, 0, :, :]
        
        boundary = get_axial_boundary_from_segmentation(seg, pixel_spacings[i], throw=int(100 / 512 * seg_img_size), tol=0.2, max_mm=100 / 512 * seg_img_size)
        boundary_list.append(boundary)
    boundary_list = torch.stack(boundary_list, axis=0) * (512. / seg_img_size)
    return boundary_list

In [14]:

def predict_det(x, model):
    
    bboxes = model(x) # N x 1 x 6
    
    return bboxes[:, 0, :]
    
    
def crop_resize_images(imgs_tensor, boundary_list, img_size=512):
    croped_list = []
    for i in range(imgs_tensor.shape[0]):
        xmin, ymin, xmax, ymax = boundary_list[i, :]
        xmin, ymin, xmax, ymax = int(xmin), int(ymin), int(xmax), int(ymax)
        # print(xmin, ymin, xmax, ymax)
        croped = TF.crop(imgs_tensor[i, :, :, :], top=ymin, left=xmin, height=ymax-ymin, width=xmax-xmin)
        croped = TF.resize(croped, (img_size, img_size))
        croped_list.append(croped)
        
    return torch.stack(croped_list, 0)

def get_original_bbox(bbox, boundary):
    scale = 512. / (boundary[:, [2]] - boundary[:, [0]])
    
    org_bbox = bbox / scale
    org_bbox[:, 0] += boundary[:, 0]
    org_bbox[:, 1] += boundary[:, 1]
    org_bbox[:, 2] += boundary[:, 0]
    org_bbox[:, 3] += boundary[:, 1]
    
    return org_bbox



In [15]:
def get_bbox_class(seg, bbox):
    """
    label 은 0.125 의 단위로, 
    seg: H x W
    bbox: [xmin, ymin, xmax, ymax]
    """
    xmin, ymin, xmax, ymax = bbox.int()
    area = seg[ymin:ymax, xmin:xmax]
    
    # print(area)
    result = torch.mean(area[area>0])
    result = torch.round(result / 0.125)
    
    return result

In [16]:
def get_bbox_class_list(seg_list, seg_bboxes):
    class_list = []
    for i in range(seg_list.shape[0]):
        class_index = get_bbox_class(seg_list[i, :, :], seg_bboxes[i, :])
        class_list.append(class_index)
        
    return torch.stack(class_list)

In [17]:

def get_class_score(scores, class_list, eps=1e-2):
    result = scores.new_zeros((scores.shape[0], 8)) + eps
    class_list = torch.nan_to_num(class_list).long()
    result[torch.arange(scores.shape[0]), class_list] = scores
    
    return result

def check_detection_result(det_result, img_size=512., threshold=0.2):
    # throw big bboxes
    areas = (det_result[:, 2] - det_result[:, 0]) * (det_result[:, 3] - det_result[:, 1]) / (img_size * img_size)
    # print(areas)
    big_indices = torch.argwhere(areas > threshold)
    det_result[big_indices, 4] = 0.
    return det_result


In [18]:
def cal_loss(prob, label):
    
    pos_weight = np.array([14, 2, 2, 2, 2, 2, 2, 2])
    neg_weight = np.array([7, 1, 1, 1, 1, 1, 1, 1])
    
    score = pos_weight * label * np.log(prob) + neg_weight * (1 - label) * np.log(1 - prob)
    
    weight_total = pos_weight * label + neg_weight * (1 - label)
    
    return -score.sum(axis=1) / weight_total.sum(axis=1)

In [19]:
def predict():
    
    with torch.no_grad():
        predictions = []
        
#         tqdm_iter = tqdm(dl)
        for (x, pixel_spacings) in tqdm(dl):
            
            # x : N x 3 x 512 x 512
            x = x.to(device)
            
            batch_probs = x.new_zeros((x.shape[0], 8)) + 1e-2
            
            seg_result = predict_seg(x, seg_model)  # N x 1 x 256 x 256
            
            active_indices = seg_result.sum(axis=[1, 2, 3]).nonzero().reshape(-1)
            if active_indices.numel() == 0:
                predictions.append(batch_probs)
                continue
            
#             tqdm_iter.set_description(f'{active_indices.numel()}')
            if active_indices.numel() != batch_size:
                x = x[active_indices, :, :, :]
                seg_result = seg_result[active_indices, :, :, :]
                pixel_spacings = pixel_spacings[active_indices]
            
            
            axial_boundary = get_axial_boundary(seg_result, pixel_spacings, seg_img_size=256)  # N x 4
            
#             print(x.shape)
            x = crop_resize_images(x, axial_boundary) # N x 3 x 512 x 512 croped
            det_result = predict_det(x, det_model)
#             det_result = check_detection_result(det_result, threshold=0.1)
            
            bboxes, scores = get_original_bbox(det_result[:, :4], axial_boundary), det_result[:, 4]
#             print(scores)
            class_list = get_bbox_class_list(seg_result[:, 0, :, :], bboxes / 2)
#             print(class_list)
            probs = get_class_score(scores, class_list)
            
            batch_probs[active_indices, :] = probs
#             print(probs)
            predictions.append(batch_probs)
        
        return torch.concat(predictions).cpu().numpy()
predictions = predict()
# predictions

100%|██████████| 83/83 [00:59<00:00,  1.40it/s]


In [20]:
df_effnet_pred = pd.DataFrame(
    data=predictions, columns=['patient_overall'] + [f'C{i}' for i in range(1, 8)]
)
df_test_pred = pd.concat([df_test_slices, df_effnet_pred], axis=1).sort_values(['StudyInstanceUID', 'Slice'])
df_patient_pred = df_test_pred.groupby('StudyInstanceUID').apply(lambda df: df.max())
df_patient_pred["patient_overall"] = df_patient_pred[[f'C{i}' for i in range(1, 8)]].max(axis=1)
df_patient_pred = df_patient_pred[['patient_overall'] + [f'C{i}' for i in range(1, 8)]]
df_patient_pred

,patient_overall,C1,C2,C3,C4,C5,C6,C7
StudyInstanceUID,,,,,,,,
1.2.826.0.1.3680043.22327,0.574853,0.070553,0.037450,0.033594,0.070707,0.189582,0.574853,0.126512
1.2.826.0.1.3680043.25399,0.508683,0.039090,0.016025,0.508683,0.252099,0.012320,0.019680,0.117002
1.2.826.0.1.3680043.5876,0.127817,0.127817,0.024592,0.035604,0.126440,0.023032,0.020248,0.029631


In [21]:
# prob = df_patient_pred.values
# label = train_df.loc[df_patient_pred.index].values

# losses = cal_loss(prob, label)
# print(list(losses))
# print(losses)
# print(np.mean(losses))
# list(losses)


In [22]:
# df = pd.read_csv('../input/rsna-2022-cervical-spine-fracture-detection/test.csv')
df_test = pd.read_csv(f'../input/rsna-2022-cervical-spine-fracture-detection/test.csv')

if df_test.iloc[0].row_id == '1.2.826.0.1.3680043.10197_C1':
    # test_images and test.csv are inconsistent in the dev dataset, fixing labels for the dev run.
    df_test = pd.DataFrame({
        "row_id": ['1.2.826.0.1.3680043.22327_C1', '1.2.826.0.1.3680043.25399_C1', '1.2.826.0.1.3680043.5876_patient_overall'],
        "StudyInstanceUID": ['1.2.826.0.1.3680043.22327', '1.2.826.0.1.3680043.25399', '1.2.826.0.1.3680043.5876'],
        "prediction_type": ["C1", "C1", "patient_overall"]}
    )

df_test

,row_id,StudyInstanceUID,prediction_type
0,1.2.826.0.1.3680043.22327_C1,1.2.826.0.1.3680043.22327,C1
1,1.2.826.0.1.3680043.25399_C1,1.2.826.0.1.3680043.25399,C1
2,1.2.826.0.1.3680043.5876_patient_overall,1.2.826.0.1.3680043.5876,patient_overall


In [23]:
df_sub = df_test.copy()
df_sub = df_sub.set_index('StudyInstanceUID').join(df_patient_pred)
df_sub['fractured'] = df_sub.apply(lambda r: r[r.prediction_type], axis=1)
df_sub

,row_id,prediction_type,patient_overall,C1,C2,C3,C4,C5,C6,C7,fractured
StudyInstanceUID,,,,,,,,,,,
1.2.826.0.1.3680043.22327,1.2.826.0.1.3680043.22327_C1,C1,0.574853,0.070553,0.037450,0.033594,0.070707,0.189582,0.574853,0.126512,0.070553
1.2.826.0.1.3680043.25399,1.2.826.0.1.3680043.25399_C1,C1,0.508683,0.039090,0.016025,0.508683,0.252099,0.012320,0.019680,0.117002,0.039090
1.2.826.0.1.3680043.5876,1.2.826.0.1.3680043.5876_patient_overall,patient_overall,0.127817,0.127817,0.024592,0.035604,0.126440,0.023032,0.020248,0.029631,0.127817


In [24]:
df_sub[['row_id', 'fractured']].to_csv('submission.csv', index=False)